# Parallel multiprocessed data acqusition and dumping to file test

In [2]:
from enum import IntFlag # Для декодирования битовых флагов

import datetime as dt
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import MetaTrader5 as mt5


# from pandas.plotting import register_matplotlib_converters
# register_matplotlib_converters()

# выведем данные о пакете MetaTrader5
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.37


**Ticker**

https://www.moex.com/ru/derivatives/equity/indices/  
Site with actual RTS futures ticker.

In [2]:
ticker = 'RIU2'

In [3]:
%%html
<style>
table {float:left}
</style>

In [4]:
# terminal init
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
else:
    print(mt5.last_error())

(1, 'Success')


The idea is not to miss any market book data and dump it to file in parallel.  
Maybe a variable with that data should be emptied every hour or upon reaching a 1 gb in memory.

In [78]:
import multiprocessing
import time
import datetime

def create_fake_data(times):
    counter = times
    df = pd.DataFrame()
    previous_end_time = datetime.datetime.now().time()
    while counter > 0:
        time.sleep(0.01)
        
        start = datetime.datetime.now().time()
        data = np.random.rand(40,10)
        data = pd.DataFrame(data)
        data['previous_end_time'] = previous_end_time
        data['start'] = start
        df = pd.concat([df,data])
        previous_end_time = datetime.datetime.now().time()
        counter -= 1
    return df

In [79]:
%%time
temp_ = create_fake_data(1000)

temp_['previous_end_time'] = pd.to_datetime(temp_['previous_end_time'],format='%H:%M:%S.%f')
temp_['start'] = pd.to_datetime(temp_['start'],format='%H:%M:%S.%f')

temp_['timedelta'] = temp_['start'] - temp_['previous_end_time']

In [93]:
temp_.memory_usage().sum()

4480000

In [90]:
temp_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40000 entries, 0 to 39
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   0                  40000 non-null  float64        
 1   1                  40000 non-null  float64        
 2   2                  40000 non-null  float64        
 3   3                  40000 non-null  float64        
 4   4                  40000 non-null  float64        
 5   5                  40000 non-null  float64        
 6   6                  40000 non-null  float64        
 7   7                  40000 non-null  float64        
 8   8                  40000 non-null  float64        
 9   9                  40000 non-null  float64        
 10  previous_end_time  40000 non-null  datetime64[ns] 
 11  start              40000 non-null  datetime64[ns] 
 12  timedelta          40000 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(10), timedelta64[ns

In [101]:
def create_fake_data(conn,times):
    counter = times
    previous_end_time = datetime.datetime.now().time()
    while counter > 0:
        df = pd.DataFrame()
        while temp_.memory_usage().sum() > 10000000:
            start = datetime.datetime.now().time()
            data = np.random.rand(40,10)
            data = pd.DataFrame(data)
            data['previous_end_time'] = previous_end_time
            data['start'] = start
            df = pd.concat([df,data])
            previous_end_time = datetime.datetime.now().time()
            print('df_created')
        conn.send(df)
        counter -= 1
    conn.send('end')
    print('conn close')
    conn.close()
    
def save_to_csv(conn):
    counter = 0
    while 1:
        df = conn.recv()
        if type(df) == str:
            print('save_done')
            break
        df.to_csv('/test/'+str(counter)+'.csv')
        print('saved')
        counter += 1
        
parent_conn, child_conn = multiprocessing.Pipe()

# creating new processes
p1 = multiprocessing.Process(target=create_fake_data, args=(parent_conn,10000))
p2 = multiprocessing.Process(target=save_to_csv, args=(child_conn,))

# running processes
p1.start()
p2.start()

# wait until processes finish
p1.join()
p2.join()

In [96]:
str_ = 'end'
type(str_) == str

True

1. Создает данные пока их не станет 10 мб
2. Отправляет
3. Сбрасывает начинает заново
4. Так столько раз, сколько указано

In [ ]:

import multiprocessing
  
def sender(conn, msgs,times):
    """
    function to send messages to other end of pipe
    """
    for msg in msgs:
        conn.send(msg)
        print("Sent the message: {}".format(msg))
    conn.close()
  
def receiver(conn):
    """
    function to print the messages received from other
    end of pipe
    """
    while 1:
        msg = conn.recv()
        if msg == "END":
            break
        print("Received the message: {}".format(msg))
  
if __name__ == "__main__":
    # messages to be sent
    msgs = ["hello", "hey", "hru?", "END"]
  
    # creating a pipe
    parent_conn, child_conn = multiprocessing.Pipe()
  
    # creating new processes
    p1 = multiprocessing.Process(target=sender, args=(parent_conn,msgs))
    p2 = multiprocessing.Process(target=receiver, args=(child_conn,))
  
    # running processes
    p1.start()
    p2.start()
  
    # wait until processes finish
    p1.join()
    p2.join()

In [5]:
# Эту штуку нужно как-нибудь запустить на день
# С начала торгового дня до конца
columns = ['type','price','volume','volume_dbl']

df_mb = pd.DataFrame()

mt5.market_book_add(ticker)

counter = 0

while dt.datetime.now().hour < 13:
#     print(counter)
    df_market_book = pd.DataFrame(mt5.market_book_get(ticker),columns=columns)
    df_market_book['time'] = dt.datetime.now()
    df_market_book['batch_number'] = counter
    df_mb = pd.concat([df_mb,df_market_book])
    counter += 1    
    time.sleep(0.05)
    
mt5.market_book_release(ticker)

df_mb

""


In [6]:
# df_mb.info()
# df_mb.to_csv('data/market_book.csv')
# df_mb['time'].describe(datetime_is_numeric=True)

In [5]:
# Ф-ция разового запроса стакана
# Здесь каждый раз идет регистрация "сбора инфы о стакане" с помощью
# mt5.market_book_add(symbol),
# а затем отзыв
# mt5.market_book_release(symbol)
def get_market_book(symbol):
    mt5.market_book_add(symbol)
    columns = ['type','price','volume','volume_dbl']
    df_market_book = pd.DataFrame(mt5.market_book_get(symbol),columns=columns)
    df_market_book['time'] = dt.datetime.now()

    mt5.market_book_release(symbol)
    return df_market_book

get_market_book(ticker)

,type,price,volume,volume_dbl,time


In [ ]:
# terminal shutdown MetaTrader 5
mt5.shutdown()